In [11]:
# Example of easy download of content.

import wikipedia
temp2 = wikipedia.page(u'B\xe1nh m\xec')
#print "pageid: {}".format(temp2.pageid)
#print "text: " +temp2.content
temp2.sections

[]

In [9]:
from database_connections import connect_to_postgres

def add_category_to_database (category_id, category_name):
    connection ,cursor = connect_to_postgres(location = 'local')
    insert_category_sql = """
    INSERT INTO category
    (category_id, category_name) VALUES ({}, {});
    """.format(category_id,category_name)
    cursor.execute(insert_category_sql)
    connection.commit()
    connection.close()
    
def add_page_to_database (page_id, page_name, category_id, page_text, section_head_array=None):
    connection ,cursor = connect_to_postgres(location = 'local')
    if section_head_array==None:
        section_head_array=[]
        
    insert_page_sql = """
    INSERT INTO page 
    (page_id, category_id, title, page, section_head) 
    VALUES ({}, {}, {}, {},{});
    """.format(page_id, page_name, category_id, page_text, section_head_array=None)
    cursor.execute(insert_category_sql)
    connection.commit()
    connection.close()
    


In [8]:
import pandas as pd
import sqlalchemy
from database_connections import connect_to_postgres
import unidecode
import yaml
import json
from requests import get
from urllib import quote
import numpy as np
from bs4 import BeautifulSoup

In [10]:
#Pipeline for custom wikipedia search.

def create_query_param_string(params):
    param_list = [key+'='+value for key, value in params.items()]
    return '?'+'&'.join(param_list)
def wikipedia_page_format(page):
    return page.replace(' ','_')

def parse_pages_from_json(response_json):
    return response_json['query']['pages']

def parse_headings_from_json(response_json):
    return response_json['mobileview']['sections']

def wikipedia_get_cagegory(category):
    params = { 'action' : 'query',
               'format' : 'json',
               'prop' : 'extracts',
               'exlimit' : 'maxl'
             }
    params['titles'] = 'Category:'+quote(category)
    query_param_string = create_query_param_string(params)
    response = get(base_url+query_param_string)
    
    try:
        return parse_pages_from_json(response.json())
    except:
        return response

def wikipedia_get_pages_for_category(category):
    params = { 'action' : 'query',
               'format' : 'json',
               'list' : 'categorymembers',
               'cmlimit' : 'max'
             }
    
    params['cmtitle'] = 'Category:'+wikipedia_page_format(category)
    query_param_string = create_query_param_string(params)
    response = get(base_url+query_param_string)
    
    return response.json()['query']['categorymembers']
    
    try:
        return parse_headings_from_json(response.json())
    except:
        return response
    
def wikipedia_get_page(title, category=False):
    """ Get the text of a wikipedia page"""
    
    params = { 'action' : 'query',
               'format' : 'json',
               'prop' : 'extracts',
               'exlimit' : 'maxl'
             }
    
    if category:
        title = "Category:"+title
        
    params['titles'] = wikipedia_page_format(title)
    query_param_string = create_query_param_string(params)
    response = get(base_url+query_param_string)
    
    return response.json()['query']['pages']

base_url = "https://en.wikipedia.org/w/api.php"

In [ ]:
category_dict = wikipedia_get_cagegory('sandwiches')

In [ ]:
category_number = int(unidecode.unidecode(category_dict.keys()[0]))

In [ ]:
category_number

In [ ]:
wikipedia_get_pages_for_category('sandwiches')

In [ ]:
temp = wikipeida.page('Hasan al-Kharrat')

In [15]:
def wikipedia_get_page_sections(title):
    """ Get the section heads of a wikipedia page"""
    
    params = { 'action' : 'query',
               'format' : 'json',
               'prop' : 'extracts',
               'exlimit' : 'maxl'
             }
        
    params['titles'] = wikipedia_page_format(title)
    query_param_string = create_query_param_string(params)
    response = get(base_url+query_param_string)
    
    return response.json()['query']['pages']

In [16]:
wikipedia_get_page_sections('Hasan al-Kharrat')

{u'39024575': {u'extract': u'<p><b>Abu Muhammad Hasan al-Kharrat</b> (Arabic: <span lang="ar" dir="rtl" xml:lang="ar">\u062d\u0633\u0646 \u0627\u0644\u062e\u0631\u0627\u0637</span>\u200e\u200e <i>\u1e24assan al-Kharr\u0101\u1e6d</i>; 1861\u201325 December 1925) was one of the principal Syrian rebel commanders of the Great Syrian Revolt against the French Mandate. His main area of operations was in Damascus and its Ghouta countryside. He was killed in the struggle and is considered a hero by Syrians.</p>\n<p>As the <i>qabaday</i> (local youth boss) of the al-Shaghour quarter of Damascus, al-Kharrat was connected with Nasib al-Bakri, a nationalist from the quarter\'s most influential family. At al-Bakri\'s invitation, al-Kharrat joined the revolt in August 1925 and formed a group of fighters from al-Shaghour and other neighborhoods in the vicinity. He led the rebel assault against Damascus, briefly capturing the residence of French High-Commissioner Maurice Sarrail before withdrawing ami